# Digit Recognizer using a Convolutional Neural Network

In [29]:
import matplotlib.pyplot as plt
import numpy as np
from keras.layers.normalization import BatchNormalization
import pandas as pd
import seaborn as sns
%matplotlib inline
# Read in the data.
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [31]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Conv2D , MaxPooling2D

labels = train.label.values.astype('int32')
x_train = train.iloc[:,1:].values.astype('float32')
x_test = test.values.astype('float32')

# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(labels)
# reshape the training and test data to feed the neural network
x_train = x_train.reshape(x_train.shape[0], 28, 28,1)
x_test = x_test.reshape(x_test.shape[0], 28, 28,1)

In [32]:
# pre-processing: divide by max 
scale = np.max(x_train)
x_train /= scale
x_test /= scale

In [34]:
from keras.layers import MaxPool2D
#Initialize the model with four convolutional layers and a pooling layer between each two convolutional layers 
# to reduce spatial data
model = Sequential()

# Add an input layer 
model.add(Conv2D(filters = 32, kernel_size = (4,4),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (4,4),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [37]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

In [36]:
from keras.preprocessing.image import ImageDataGenerator
#generate more images
gen = ImageDataGenerator(rotation_range=10, 
                               width_shift_range=0.10, 
                               shear_range=0.5,
                               height_shift_range=0.10, 
                               zoom_range=0.10
                              )

In [38]:
# Fit the model on the generated Data
history = model.fit_generator(gen.flow(x_train,y_train, batch_size=32),
                              epochs = 15, verbose = 1, steps_per_epoch=x_train.shape[0] // 32
                              )

Epoch 1/15
1312/1312 [==============================] - 18s 13ms/step - loss: 0.3217 - acc: 0.8976
Epoch 2/15
1312/1312 [==============================] - 17s 13ms/step - loss: 0.1177 - acc: 0.9651
Epoch 3/15
1312/1312 [==============================] - 17s 13ms/step - loss: 0.0931 - acc: 0.9724
Epoch 4/15
1312/1312 [==============================] - 21s 16ms/step - loss: 0.0782 - acc: 0.9766
Epoch 5/15
1312/1312 [==============================] - 24s 18ms/step - loss: 0.0682 - acc: 0.9804
Epoch 6/15
1312/1312 [==============================] - 23s 18ms/step - loss: 0.0610 - acc: 0.9824
Epoch 7/15
1312/1312 [==============================] - 23s 18ms/step - loss: 0.0585 - acc: 0.9832
Epoch 8/15
1312/1312 [==============================] - 23s 17ms/step - loss: 0.0560 - acc: 0.9837
Epoch 9/15
1312/1312 [==============================] - 23s 18ms/step - loss: 0.0520 - acc: 0.9840
Epoch 10/15
1312/1312 [==============================] - 24s 18ms/step - loss: 0.0510 - acc: 0.9854
Epoch 11/

In [40]:
#fit the model wiht the real Data
model.fit(x_train, y_train,epochs=5, batch_size=16,validation_split=0.1, verbose=1)

Train on 37800 samples, validate on 4200 samples
Epoch 1/5
37800/37800 [==============================] - 34s 890us/step - loss: 0.0350 - acc: 0.9898 - val_loss: 0.0166 - val_acc: 0.9950
Epoch 2/5
37800/37800 [==============================] - 35s 920us/step - loss: 0.0328 - acc: 0.9901 - val_loss: 0.0128 - val_acc: 0.9952
Epoch 3/5
37800/37800 [==============================] - 36s 959us/step - loss: 0.0313 - acc: 0.9913 - val_loss: 0.0328 - val_acc: 0.9900
Epoch 4/5
37800/37800 [==============================] - 36s 943us/step - loss: 0.0301 - acc: 0.9908 - val_loss: 0.0265 - val_acc: 0.9926
Epoch 5/5
37800/37800 [==============================] - 36s 952us/step - loss: 0.0291 - acc: 0.9917 - val_loss: 0.0144 - val_acc: 0.9943


In [41]:
# I got a pretty good results 0.994 on the kaggle competition
y_pred = model.predict_classes(x_test)
pd.DataFrame({'ImageId': range(1,28001), 'Label': y_pred}).to_csv('results.csv', index=False)